__NAME:__ __FULLNAME__  
__SECTION:__ __NUMBER__  
__CS 5970: Machine Learning Practices__

# Homework 12: Dimensionality Reduction with Isometric Mapping

## Assignment Overview
Follow the TODOs and read through and understand any provided code.  
For all plots, make sure all necessary axes and curves are clearly and 
accurately labeled. Include figure/plot titles appropriately as well. Post and questions you may have to the Canvas Discussion.

### Task
For this assignment you will be exploring dimensionality reduction using IsoMap.
IsoMap creates a graph of nearest neighbors whilst attempting to perserve geodesic distances along a manifold.

### Data set
The data set is synthetic.  
File: hw12_isomap_squirrelly_data.csv  
This data set has 1000 samples and four columns (index, x0, x1, x2).   
Your goal is to predict x2 given x0 and x1 using the LinearRegression model.  


### Objectives
* Overfitting
* Regularization
* Dimensionality Reduction with Isomap

### Notes
* Do not save work within the ml_practices folder

### General References
* [Guide to Jupyter](https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook)
* [Python Built-in Functions](https://docs.python.org/3/library/functions.html)
* [Python Data Structures](https://docs.python.org/3/tutorial/datastructures.html)
* [Numpy Reference](https://docs.scipy.org/doc/numpy/reference/index.html)
* [Numpy Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf)
* [Summary of matplotlib](https://matplotlib.org/3.1.1/api/pyplot_summary.html)
* [DataCamp: Matplotlib](https://www.datacamp.com/community/tutorials/matplotlib-tutorial-python?utm_source=adwords_ppc&utm_campaignid=1565261270&utm_adgroupid=67750485268&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=1t1&utm_creative=332661264365&utm_targetid=aud-299261629574:dsa-473406587955&utm_loc_interest_ms=&utm_loc_physical_ms=9026223&gclid=CjwKCAjw_uDsBRAMEiwAaFiHa8xhgCsO9wVcuZPGjAyVGTitb_-fxYtkBLkQ4E_GjSCZFVCqYCGkphoCjucQAvD_BwE)
* [Pandas DataFrames](https://urldefense.proofpoint.com/v2/url?u=https-3A__pandas.pydata.org_pandas-2Ddocs_stable_reference_api_pandas.DataFrame.html&d=DwMD-g&c=qKdtBuuu6dQK9MsRUVJ2DPXW6oayO8fu4TfEHS8sGNk&r=9ngmsG8rSmDSS-O0b_V0gP-nN_33Vr52qbY3KXuDY5k&m=mcOOc8D0knaNNmmnTEo_F_WmT4j6_nUSL_yoPmGlLWQ&s=h7hQjqucR7tZyfZXxnoy3iitIr32YlrqiFyPATkW3lw&e=)
* [Sci-kit Learn Linear Models](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model)
* [Sci-kit Learn Ensemble Models](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble)
* [Sci-kit Learn Metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)
* [Sci-kit Learn Model Selection](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection)
* [Sci-kit Learn Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)
* [Sci-kit Learn Preprocessing](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)

In [ ]:
import sys

import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import os, re, fnmatch
import pathlib, itertools
import time as timelib
import matplotlib.pyplot as plt
import matplotlib.patheffects as peffects

from matplotlib import cm
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import explained_variance_score, confusion_matrix
from sklearn.metrics import mean_squared_error, roc_curve, auc, f1_score
from sklearn.externals import joblib
from sklearn.manifold import Isomap

FIGW = 15
FIGH = 4
FONTSIZE = 10

#plt.rcParams['figure.figsize'] = (FIGW,FIGH)
plt.rcParams['font.size'] = FONTSIZE

plt.rcParams['xtick.labelsize'] = FONTSIZE
plt.rcParams['ytick.labelsize'] = FONTSIZE

%matplotlib inline
#https://matplotlib.org/3.1.1/tutorials/introductory/images.html
plt.style.use('ggplot')

In [ ]:
"""
Display current working directory of this notebook. If you are using 
relative paths for your data, then it needs to be relative to the CWD.
"""
HOME_DIR = pathlib.Path.home()
pathlib.Path.cwd()

# LOAD DATA

In [ ]:
"""
Load the data
"""
data = pd.read_csv('hw12_isomap_squirrelly_data.csv')

nrows, ncols = data.shape
print("%d rows and %d columns" % (nrows, ncols))

In [ ]:
""" TODO
Seperate the feature inputs from the predicted output
x0 and x1 are the inputs
x2 is the output we want to predict
"""
# TODO: Get x0 and x1 as the inputs


# TODO: Get x2 as the output



In [ ]:
""" PROVIDED
Visualize the data
"""
# Position along the manifold
pos = np.arange(0, 1, .001)

fig = plt.figure(figsize=(FIGW,20)) #plt.figaspect(0.5))

# 3D View
ax0 = fig.add_subplot(3, 1, 1, projection='3d')
ax0.scatter(X['x0'], X['x1'], x2)
ax0.set(xlabel='x0', ylabel='x1', zlabel='x2')
ax0.set_title('3D View')

# Top Down View
ax1 = fig.add_subplot(3, 1, 2)
ax1.plot(X['x0'], X['x1'], 'm.')
ax1.set(xlabel='x0', ylabel='x1')
ax1.set_title('Top Down View')

# View Along the Data Manifold
ax2 = fig.add_subplot(3, 1, 3)
ax2.plot(pos, x2, 'b.')
ax2.set(ylabel='x2')
ax2.set_title('View Along the Manifold')

In [ ]:
""" PROVIDED
The unnamed column is the row index
"""
data.head(3)

### PRELIMINARY DISCUSSION
Describe the behavior of these data in 3 to 4 sentences.

# LINEAR REGRESSION

In [ ]:
""" TODO
Fit a linear model using X as the input
and the position along the manifold, the variable pos, as the output
"""
# TODO: create and fit the linear model. The output is pos
lnr = # TODO


# TODO: predict the positions from the original input X with the linear model
pos_preds = # TODO: predict

# TODO: Show the results. compare the true positions to the predictions
fig, ax = plt.subplots(figsize=(FIGW,FIGH))
ax.plot(pos, ''' TODO ''', 'b', label='True Position')
ax.plot(pos, ''' TODO ''', 'r', label='Prediction')
ax.set(xlabel='Position', ylabel='Output')
ax.legend()

# ISOMAP

In [ ]:
""" TODO
Fit an IsoMap object to the input data X, which contains the 
features x0 and x1
Use the IsoMap object to transform the input data
try small values, less than 10, for the number of neighbors and 
think about reasonable values for n_components based on the data
"""
# TODO: create and fit an IsoMap object 
isomap = # TODO


# TODO: transform the inputs with the IsoMap object
Xmap = # TODO

# Plot the result overlaying the True Positions
# with the IsoMap transformed features
fig, ax = plt.subplots(figsize=(FIGW,FIGH))
ax.plot(pos, 'b', label='True Position')
ax.plot(Xmap, 'r', label='Mapping')
ax.legend()

In [ ]:
""" TODO
Fit another linear regression model on the
isomap transformed inputs and the position 
along the manifold, the variable pos, as the output
"""
# TODO: Create and fit the model to the IsoMap features
lnr_isomap = # TODO


# TODO: predict the positions from the IsoMap features, with the linear model
pos_preds_isomap = # TODO

# TODO: Show the prediction results, overlayed with the true positions



# POLYNOMIAL FEATURES

In [ ]:
""" TODO
Construct polynomial expanded features from the original X input
using PolynomialFeatures. Play with a few values for the degree 
between 2 and 6, and set include_bias to True

Fit another linear regression model from the polynomial expanded
features as input and the position along the manifold, the variable 
pos, as the output
"""
# TODO: Construct the polynomial features from the original inputs
poly_features1 = # TODO
Xpoly = # TODO: transform the original inputs

# TODO: Fit the model to the polynomial features
lnr_poly = # TODO: create and fit the model
pos_preds_poly = # TODO: predict using the polynomial transformed features

# TODO: Show the results. compare the true positions to the predictions



In [ ]:
""" TODO
Construct polynomial features from the IsoMap transformed
input features

Fit another linear regression model from these features as input
and the position along the manifold, the variable pos, as the output
"""
# TODO: Construct the polynomial features from the IsoMap inputs
poly_features2 = # TODO
Xmap_poly = # TODO: transform the Isoap features

# TODO: Fit the model to the polynomial IsoMap features
lnr_iso_poly = # TODO: create anf fit linear model
pos_preds_iso_poly = # TODO: predict with the poly-isomap features

# TODO: Show the results. Compare position to the predicted output



# PREDICITON

### LinearRegression Benchmark
In the previous section, we were trying to predict the position along the manifold from various forms of the inputs. Now you will try to predict actual position in 3D space, x2.

In [ ]:
""" TODO
Fit a linear regression model that predicts x2 from the original x0 and x1
output is x2
"""
# TODO: Fit the model to the original features. The output is x2
lnr = # TODO
x2preds = # TODO: compute the predictions from X

# TODO: Show the results. Compare the predicitons to the true x2



# TODO: Compute the training/prediction error. Use mean_squared_error()



### IsoMap LinearRegression

In [ ]:
""" TODO
Fit a linear regression model to predict x2 from the IsoMap
transformed versions of x0 and x; output is x2
"""
# TODO: Fit the model to the IsoMap features computed above. output is x2
lnr_isomap = # TODO: Create and fit the model
x2preds_isomap = # TODO: predict with the IsoMap features

# TODO: Show the results. Compare the predicitons to the true x2


# TODO: Compute the training/prediction error. Use mean_squared_error()



### PolynomialFeatures LinearRegression

In [ ]:
""" TODO
Fit a linear regression model to predict x2 from the polynomially
expanded features; output is x2
"""
# TODO: Construct the polynomial features from the original inputs
poly_features1 = # TODO: 
Xpoly = # TODO: construct the polynomial expansion of the original inputs
print(Xpoly.shape)

# TODO: Fit the model to the polynomial features. output is x2
lnr_poly = # TODO: Create and fit linear model
x2preds_poly = # TODO: predict using the polynomial features

# TODO: Show the results. Compare the predictions to the true x2



# TODO: Compute the training error



### IsoMap Polynomial Features LinearRegression

In [ ]:
""" TODO
Fit a linear regression model to predict x2 from the polynomially
expanded IsoMap transformed features; output is x2
"""
# TODO: Construct the polynomial features from the IsoMap inputs
poly_features2 = # TODO
Xmap_poly = # TODO: transform the IsoMap features

# TODO: Fit the model to the polynomial IsoMap features. output is x2
lnr_iso_poly = # TODO: create and fit model
x2preds_iso_poly = # TODO: predict from polynomial IsoMap features

# TODO: Show the results. Compare true x2 to the predictions


# TODO: Compute the training error



# DISCUSSION
In several paragraphs, compare the performance of each of the Linear Regression models explored within the PREDICTION section. List the models in order of the quality of the predictions (1 being the model with the best quality). Is the model with the best quality perfect? Please be clear and concise in your explanation. 